# Requirements Gathering & Analysis
---

## Requirements

### Functional Requirements

The program should: 
1.  Accept as input a YouTube URL.
2.  Validate the input to ensure validity. 
3.  Extract the video ID from the given URL.
4.  Fetch the transcript for the given URL.
5.  Format the fetched transcript into a readable format. 
6.  Additionally accept as input a filename. 
7.  Save the formatted transcript to the given filename. 

### Nonfunctional Requirements

1. Performance
    - Reasonable time to fetch and save transcripts.
2. Reliability
    - Gracefully handle errors.
3. Security
    - Secure handling of user input. 
    - Not violating terms of service. 
4. Portability
    - Platform-independent. 
5. Maintainability
    - Code should be well-structured, commented to allow for updates/fixes as required.
6. Scalability
    - The program should be able to handle larger requests (longer videos) without significant performance degredation. 

### Technical Requirements

- IDE (installed and baselined)
    - Python
- Python Standard Library
- youtube_transcript_api
- Requests library

Developer to possess an understanding of:
- JSON data structures
- File handling in Python

## Problem

### Real-World Application

**About**

- I lack the auditory processing skills required to retain information presented in a YouTube video. However I enjoy content from several creators on the platform. 
  - I prefer to read the content, much as one might read an article or a book. Unfortunately, the process of making a copy of the video transcript is somewhat cumbersome, and not especially time-efficient.

**Accessibility**

- I hope to create a program to make copies of specified video transcripts more accessible. 

### What Does Success Look Like?

1. Functionality
    - The program successfully extracts the transcript from a given YouTube video URL. This is the primary function of the program, so its ability to do this correctly would be a key measure of success.

2. Accuracy
    - The extracted transcript accurately represents the content of the video.
    - Potentially measured by comparing a sample of the extracted transcript to the actual video content and checking for discrepancies.

3. Efficiency
    - The program performs the extraction process within an acceptable time frame. While the exact time may depend on factors like the length of the video, the program should not take an excessively long time to perform the extraction.

4. Usability
    - The program is easy to use. A user with reasonable computer skills should be able to use the program without excessive difficulty.
    - Potentially measured through user testing and feedback.

5. Reliability
    - The program can handle different types of videos and does not crash or produce errors during the extraction process. It should be able to handle edge cases gracefully, such as when a video does not have a transcript available.

6. Security
    - The program does not expose the user or their data to security risks. This might involve ensuring that the program does not require unnecessary permissions, that it handles data securely, and that it does not expose the user to potential security threats from external sources.


# Design
---

## Architecture

### First Iteration
1. Given a URL, extract the video ID.
2. Given a Video ID, fetch the transcript.
3. Given fetched transcript, reformat to readable format. 
4. Given reformatted transcript, output according to requirements.
5. From function call (main), integrate preceeding functions to accept input and return desired output. 

## Exception Handling

### Areas Vulnerable to Error
    
1.  URL Parsing[`extract_video_id`]

    - **Possible Exception(s)**
        - If given URL is not a valid YouTube URL, the `extract_video_id` function may not be able to extract a valid video ID. 
    - **Solution**
        - Validate input (URL & video ID)  
<br>

2. Fetching the Transcript

    - **`get_transcript`**
        - **Possible Exception(s)**
            - Transcript not available. Some YouTube videos do not have transcripts. When the video ID of such a video is passed to the `get_transcript` function, it may result in an error. 
        - **Solution**
            - Wrap the function fetching the transcript in a try-except block.
    - **`format_transcript`**
        - **Possible Exception(s)**
            - Empty transcript - Occasionally a transcript for a given video doesn't contain any text. Returning an empty string.
        - **Solution** 
            - Check for empty string, if true - raise custom exception.  
<br>
  
3. File Handling [`output_transcript`]

    - **Possible Exception(s)**
        - As function [`output_transcript`] handles output of data to a file, in the event of a duplicate file name, the function will overwrite the file.
            - Intended behavior
    - **Solution**
        - Best practice to notify user, request input in the event of overwriting file.  
<br>

4. I/O Errors [`output_transcript`]

    - **Possible Exception(s)**
        - As function [`output_transcript`] handles output of data to a file, error is possible.
            - Lack of permissions
            - Insufficient disk space
    - **Solution**
        - Encase function `output_transcript` in a try-except block. 
<br>

### Additional Considerations
- Specificity
    - Catch only specific exceptions as you exxpect them to occur.
    - Allow unexpected exceptions to propogate.
        - Subsequently handled by a higher-level error handler.
        - Logged for debugging.

# Implementation
---

## Components
---

### 1 `extract_video_id(url)`
> *Given a URL, return a Video ID*

In [ ]:
from urllib.parse import urlparse, parse_qs

def extract_video_id(url):
  '''
  Function to extract the video ID from a YouTube URL.
  PARAMETERS:
  url (type: str): The YouTube URL
  RETURN:
  video_id 'v' (type: str): Extracted video ID, or None if no ID found.
  '''
  # Use urlparse to break up the URL into components.
  parsed_url = urlparse(url)
  
  # Use parse_qs to parse the query string (after '?') from given URL.
  # This returns a dictionary where keys are parameter names ('v' as it were)
  # and values are lists of corresponding values. 
  video_id = parse_qs(parsed_url.query).get('v')
  
  # If 'v' parameter is found in the dictionary, return it's value (video ID)
  # If not found, return None.
  if video_id:
    return video_id[0] # 'v' parameter's value is the video ID
  else:
    return None # 'v' parameter not found in given URL

### 2 `get_transcript(video_id)`
> Given the Video ID, return the Transcript.

In [ ]:
# Import the YouTubeTranscriptApi library.
# Provides functionality with YouTube to fetch transcripts
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_id):
  '''
  Function to fetch a transcript given a YouTube video ID. 
  PARAMETERS:
  video_id (type: str): YouTube video ID
  RETURNS:
  transcript (type: list of dict):
  The transcript is returned as a list of dictionaries.
  Each dictionary represents a segment of the transcript, contains text, 
  start time, and duration.
  '''
  try:
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    # If fetch was successful, return transcript as list of dictionaries.
    return transcript
    # If an error occurs during fetch, catch exception. 
  except Exception as e:
    # Print an error message. Placeholder {e} is replaced with details.
    print(f"An error occurred: {e}")
    # Since an error occurred, return type None. 
    return None

### 3 `format_transcript(transcript)`
> Given the Transcript, return reformatted transcript.

In [ ]:
def format_transcript(transcript):
  '''
  Function to format the returned transcript.
  PARAMETERS:
  transcript (list): Data from the transcript as a list of dictionaries.
  RETURNS:
  str: The formatted transcript as a single string. 
  '''

  # Use a list comprehension to extract the 'text' from each dictionary.
  # The resultant texts list is a list of strings.
  texts = [segment['text'] for segment in transcript]

  # Join the strings in the texts list into a single string, with each 
  # string separated by a space. 
  formatted_transcript = ' '.join(texts)

  return formatted_transcript

### 4 `output_transcript(transcript, filename)`
> Given reformatted transcript + filename, output the transcript to a file.

In [ ]:
def output_transcript(transcript, filename):
  '''
  Function to output the transcript to a text file. 
  PARAMETERS:
  transcript (str): Reformatted script
  filename (str): Name of file to which to write within CD
  RETURNS:
  None
  '''

  # Open the specified file in write mode ('w').
  # If file exists - overwrite. If not - create. 
  with open(filename, 'w') as file:
    # Write the transcript to the file.
    file.write(transcript)

## Integration (1-4)
> Integrate components with function:   `main`. 

In [ ]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

# Function to extract video ID from URL
def extract_video_id(url):
    '''
    Function to extract the video ID from a YouTube URL.

    Parameters:
    url (str): The YouTube URL.

    Returns:
    str: The extracted video ID.
    '''

    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the video ID from the 'v' query parameter
    video_id = parse_qs(parsed_url.query).get('v')
    if video_id:
        return video_id[0]
    else:
        return None

# Function to obtain transcript
def get_transcript(video_id):
    '''
    Function to get the transcript of a YouTube video.

    Parameters:
    video_id (str): The YouTube video ID.

    Returns:
    list: The video transcript.
    '''

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to format transcript
def format_transcript(transcript):
    '''
    Function to format the transcript.

    Parameters:
    transcript (list): The transcript data as a list of dictionaries.

    Returns:
    str: The formatted transcript as a single string.
    '''

    # Use a list comprehension to extract the 'text' from each dictionary.
    texts = [segment['text'] for segment in transcript]

    # Join the strings in the texts list into a single string.
    formatted_transcript = ' '.join(texts)

    return formatted_transcript
"""
# Function to output transcript (print)
def output_transcript(transcript):
  '''
  Function to output the transcript to the console.
  
  Parameters:
  transcript (str): The reformatted transcript.
  
  Returns:
  None
  '''
  print(transcript)
 """
# Function to output transcript (save)
def output_transcript(transcript, filename):
  '''
  Function to output the trasncript to a file. 

  Parameters:
  transcript(str): The reformatted transcript.

  Returns:
  None
  '''
  # Open the specified file in write mode ('w').
  # If file already exists - overwrite. If new - create.
  with open(filename, 'w') as file:
    # Write the transcript to the file.
    file.write(transcript)

# Main function
def main(url, filename):
    '''
    Main function to get and output the transcript of a YouTube video.

    Parameters:
    url (str): The YouTube video URL.
    filename (str): The name of the file (within cd) to which to save transcript.

    Returns:
    None
    '''

    # Extract the video ID from the URL.
    video_id = extract_video_id(url)

    # Get the transcript.
    transcript = get_transcript(video_id)

    # Format the transcript.
    formatted_transcript = format_transcript(transcript)

    # Output the transcript.
    output_transcript(formatted_transcript, filename)

# Test the main function
main("https://www.youtube.com/watch?v=dQw4w9WgXcQ", "transcript.txt")

# Testing
---

# Maintenance
---